In [1]:
import time
import pandas as pd
import numpy as np
import lightgbm as lgb
import random
import config
import re
import os
import gc
import pickle
import scipy.special as special

from math import log
from numba import jit
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import log_loss,roc_curve
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain, combinations

In [2]:
def ks_metric(true,score):
    fpr, tpr, thresholds = roc_curve(true,score)
    ks = max(tpr-fpr)
    return ks 

def score_change(score,base_rate,real_rate):
    base_change = np.log(base_rate/(1-base_rate)) - np.log(real_rate/(1-real_rate))
    score_adj = np.exp(np.log(score/(1-score)) - base_change)/(np.exp(np.log(score/(1-score)) - base_change)+1)
    return score_adj

def memory_saving(df):
    for var in df:
        if df[var].dtypes == float:
            df[var] = pd.to_numeric(df[var],downcast='float')
        else:
            df[var] = pd.to_numeric(df[var],downcast='signed')
    return df

def map_col(df,drop=False):
    map_dict = {
        'item_price_level':[4,5,6,7,8,9],
        'item_sales_level':[4,6,9,10,11,12,13,14,16],
        'item_pv_level':[6,9,10,11,12,13,14,15,16,17,18,19,20],
        'user_age_level':[1001,1002,1003,1004,1005],
        'context_page_id':[4001,4002,4004,4006,4008,4010,4013,4016,4018],
        'shop_review_num_level':[5,9,14,15,16,17,18,20,21],
        #'hour':[6,9,12,17,20],
        'user_occupation_id':{-1:2003},
        'user_star_level':{-1:3000}
    }
    for key,value in map_dict.items():
        if isinstance(value,list):
            df[key+'_mapped'] = 0
            for i in range(len(value)):
                df.loc[df[key]>value[i],key+'_mapped'] = i+1
        else:
            '''df[key+'_mapped'] = df[key]
            for key_sub,value_sub in value.items():
                df.loc[df[key]==key_sub,key+'_mapped'] = value_sub'''
            df[key+'_mapped'] = df[key].apply(lambda x:value.get(x,x))
        if drop:
            df[key] = df[key+'_mapped']
            del df[key+'_mapped']
    gc.collect()
    return df

<font color=#0099ff size=5 face="黑体">读取数据</font>

In [3]:
featureDtypes = {'cnt_rec': 'int8',
 'context_id': 'int64',
 'context_page_id': 'int16',
 'context_timestamp': 'int32',
 'day': 'int8',
 'hour': 'int8',
 'instance_id': 'int64',
 'is_trade': 'float32',
 'item_brand_id': 'int16',
 'item_category_list_bin1': 'int8',
 'item_category_list_bin2': 'int8',
 'item_city_id': 'int16',
 'item_collected_level': 'int8',
 'item_id': 'int32',
 'item_price_level': 'int8',
 'item_pv_level': 'int8',
 'item_sales_level': 'int8',
 'len_item_property_list': 'int8',
 'len_predict_category_property': 'int8',
 'min': 'int8',
 'shop_id': 'int16',
 'shop_review_num_level': 'int8',
 'shop_review_positive_rate': 'float32',
 'shop_score_delivery': 'float32',
 'shop_score_description': 'float32',
 'shop_score_service': 'float32',
 'shop_star_level': 'int16',
 'user_age_level': 'int16',
 'user_gender_id': 'int8',
 'user_id': 'int32',
 'user_occupation_id': 'int16',
 'user_star_level': 'int16'}

dfAll = pd.read_table(config.FEATURE_SET,sep=' ',dtype=featureDtypes)
dfAll = map_col(dfAll,True); gc.collect()

0

In [4]:
dataRootDir = '../../Data/advertisment/Cache/'
dataAdded = [
    'ratio_rank',
    #'ratio_rank_preday',
    'smooth',
    'offline',
    'cross_plus',
    'trick_userid',
    'text_base'
]

for add in dataAdded:
    tmpDf = pd.read_csv(dataRootDir + add +'.csv')
    tmpDf = memory_saving(tmpDf)
    dfAll = pd.concat([dfAll,tmpDf],axis=1)
    del tmpDf
    gc.collect()
    print('%s is loaded, shape is %d'%(add,dfAll.shape[1]))

ratio_rank is loaded, shape is 312
smooth is loaded, shape is 349
offline is loaded, shape is 647
cross_plus is loaded, shape is 812
trick_userid is loaded, shape is 869
text_base is loaded, shape is 875


In [5]:
with open(dataRootDir+'del_features.pkl','rb') as f:
    feat_del = pickle.load(f)
for var in feat_del:
    del dfAll[var]
print(dfAll.shape)

(1597063, 655)


<font color=#0099ff size=5 face="黑体">拆分样本</font>

In [6]:
features = [i for i in dfAll.columns.tolist() if not i in config.IGNORE_COLS+['min']]

train_idx = dfAll.loc[(dfAll['hour']<10)&(dfAll['hour']>0)].index
valid_idx = dfAll.loc[(dfAll['hour']<12)&(dfAll['hour']>9)].index
Xi_train_, y_train_ = dfAll.loc[list(train_idx),features],dfAll.loc[train_idx,'is_trade']
Xi_valid_, y_valid_ = dfAll.loc[list(valid_idx),features],dfAll.loc[valid_idx,'is_trade']
Xi_test_ = dfAll.loc[(dfAll['hour']>=12),features]

del dfAll
del train_idx
del valid_idx

<font color=#0099ff size=5 face="黑体">模型</font>

In [7]:
clf = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    num_leaves=40, 
    max_depth=8,
    n_estimators=20000,
    n_jobs=20,
    learning_rate=0.05,
    colsample_bytree=0.8,
    subsample=0.9,
    max_bin=20
)
clf.fit(Xi_train_[features], y_train_, eval_set=[(Xi_valid_[features], y_valid_)],feature_name = features,
        categorical_feature=[],early_stopping_rounds=100)
#[i for i in ['item_category_list_bin1','item_category_list_bin2'] if i in features]
y_score_ = clf.predict_proba(Xi_valid_[features],)[:, 1]

print(pd.Series(clf.feature_importances_, features).sort_values(ascending=False).reset_index())
print(log_loss(y_valid_, y_score_))
print(ks_metric(y_valid_, y_score_))
bstIter = clf.best_iteration_

C:\Users\Leo Mao\Anaconda3\lib\site-packages\lightgbm\basic.py:1029: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's binary_logloss: 0.652523
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.615726
[3]	valid_0's binary_logloss: 0.582305
[4]	valid_0's binary_logloss: 0.551872
[5]	valid_0's binary_logloss: 0.524043
[6]	valid_0's binary_logloss: 0.498525
[7]	valid_0's binary_logloss: 0.475085
[8]	valid_0's binary_logloss: 0.453519
[9]	valid_0's binary_logloss: 0.433593
[10]	valid_0's binary_logloss: 0.415184
[11]	valid_0's binary_logloss: 0.398169
[12]	valid_0's binary_logloss: 0.382383
[13]	valid_0's binary_logloss: 0.367769
[14]	valid_0's binary_logloss: 0.354174
[15]	valid_0's binary_logloss: 0.341536
[16]	valid_0's binary_logloss: 0.329804
[17]	valid_0's binary_logloss: 0.318872
[18]	valid_0's binary_logloss: 0.308688
[19]	valid_0's binary_logloss: 0.299213
[20]	valid_0's binary_logloss: 0.290352
[21]	valid_0's binary_logloss: 0.282094
[22]	valid_0's binary_logloss: 0.274396
[23]	valid_0's binary_logloss: 0.267208
[24]	valid_0's binary_logl

[203]	valid_0's binary_logloss: 0.164339
[204]	valid_0's binary_logloss: 0.16434
[205]	valid_0's binary_logloss: 0.164341
[206]	valid_0's binary_logloss: 0.164342
[207]	valid_0's binary_logloss: 0.164344
[208]	valid_0's binary_logloss: 0.164348
[209]	valid_0's binary_logloss: 0.16435
[210]	valid_0's binary_logloss: 0.164345
[211]	valid_0's binary_logloss: 0.164338
[212]	valid_0's binary_logloss: 0.16434
[213]	valid_0's binary_logloss: 0.164338
[214]	valid_0's binary_logloss: 0.164335
[215]	valid_0's binary_logloss: 0.164336
[216]	valid_0's binary_logloss: 0.164332
[217]	valid_0's binary_logloss: 0.164333
[218]	valid_0's binary_logloss: 0.164324
[219]	valid_0's binary_logloss: 0.164328
[220]	valid_0's binary_logloss: 0.16433
[221]	valid_0's binary_logloss: 0.164332
[222]	valid_0's binary_logloss: 0.16433
[223]	valid_0's binary_logloss: 0.164329
[224]	valid_0's binary_logloss: 0.164327
[225]	valid_0's binary_logloss: 0.164323
[226]	valid_0's binary_logloss: 0.164322
[227]	valid_0's binar

In [ ]:
0.16444608803
0.342700836673
0.164416256943
0.343129542574



0.164356482036
0.343516388482

0.164336989298
0.34542592653


5/2
0.164280909485
0.344169354365

0.164288423185
0.345002390217

0.164260712163
0.345363856891

0.164321791808
0.343834063361

0.16422442281
0.345099924645

0.164310490084
0.344753581033

0.164297241068
0.344992845172

0.16428814598
0.343929045439

0.164230224672
0.344283646228

In [16]:
xx = pd.Series(clf.feature_importances_, features).sort_values(ascending=False).reset_index()
(xx[0]==0).sum()

4

In [17]:
features = xx.loc[xx[0]>0,'index'].tolist()
feat_del += xx.loc[xx[0]==0,'index'].tolist()

In [18]:
import pickle
with open(dataRootDir+'del_features.pkl','wb') as f:
    pickle.dump(feat_del,f)

In [21]:
bstIter

315

In [25]:
Xi_finnal_.head()

,context_page_id,hour,item_brand_id,item_category_list_bin1,item_city_id,item_collected_level,item_id,item_sales_level,len_item_property_list,len_predict_category_property,...,user_id~item_category_list_bin1_after_exist,user_id~item_category_list_bin2_before_exist,user_id~item_category_list_bin2_after_exist,user_id~item_category_list_bin2_sametime_exist,cate_predict_chk,cate_predict_common_property,property_predict_avg_cnt,property_join_cnt,property_gap1_cnt,property_gap2_cnt
0,0,3,12674,6,118,13,57451,3,18,2,...,0,0,0,0,1,1,2.000000,0.050000,0.100000,0.850000
2,0,8,1094,19,2,14,76885,7,27,1,...,0,0,0,1,0,0,5.000000,0.066667,0.100000,0.833333
3,3,8,12224,11,157,15,68144,6,24,1,...,0,1,0,0,1,2,2.000000,0.083333,0.000000,0.916667
4,0,8,11862,2,157,14,29850,7,18,3,...,1,1,1,1,1,0,2.666667,0.045455,0.181818,0.772727
5,8,7,7271,3,157,12,79093,4,22,2,...,1,1,1,0,1,1,6.500000,0.178571,0.214286,0.607143


In [8]:
Xi_finnal_ ,y_finnal_ = pd.concat([Xi_train_,Xi_valid_]), pd.concat([y_train_,y_valid_])
del Xi_train_
del Xi_valid_
del y_train_
del y_valid_

In [10]:
#Xi_finnal_ ,y_finnal_ = np.vstack((Xi_train_,Xi_valid_),np.hstack((y_train_,y_valid_))
'''Xi_finnal_ ,y_finnal_ = pd.concat([Xi_train_,Xi_valid_]), pd.concat([y_train_,y_valid_])
del Xi_train_
del Xi_valid_'''

clf = lgb.LGBMClassifier(
    num_leaves=40, 
    max_depth=8,
    n_estimators=bstIter,
    n_jobs=20,
    learning_rate=0.05,
    colsample_bytree=0.8,
    subsample=0.9,
    max_bin=20
)
clf.fit(Xi_finnal_[features], y_finnal_,feature_name = features,
        categorical_feature=[])
#[i for i in ['item_category_list_bin1','item_category_list_bin2'] if i in features]
y_test_meta = np.zeros((Xi_test_.shape[0], 1), dtype=float)
y_test_meta[:,0] += clf.predict_proba(Xi_test_[features])[:,1]
dfinstance = pd.read_table(config.FEATURE_SET,sep=' ',dtype=featureDtypes,usecols=[4,5])
submit = pd.DataFrame({'instance_id':dfinstance.loc[dfinstance['hour']>=12,'instance_id'],'predicted_score':y_test_meta[:,0]})
#submit.to_csv('../../Submission/advertisement/gbm_trick_0330.txt', sep=" ", index=False, line_terminator='\n')

C:\Users\Leo Mao\Anaconda3\lib\site-packages\lightgbm\basic.py:1029: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [14]:
y_test_meta = np.zeros((Xi_test_.shape[0], 1), dtype=float)
y_test_meta[:,0] += clf.predict_proba(Xi_test_[features])[:,1]
dfinstance = pd.read_table(config.FEATURE_SET,sep=' ',dtype=featureDtypes,usecols=[4,5])
submit = pd.DataFrame({'instance_id':dfinstance.loc[dfinstance['hour']>=12,'instance_id'],'predicted_score':y_test_meta[:,0]})

In [15]:
print(y_finnal_.mean())
print(submit['predicted_score'].mean())

0.045298803597688675
0.04809151423883345


In [12]:
submit.to_csv('../../Submission/advertisement/lgb_trick_cross_offline_textbase_502.txt', sep=" ", index=False, line_terminator='\n')

In [13]:
submit['predicted_score'] = score_change(submit['predicted_score'],submit['predicted_score'].mean(),0.0359194123126834)
print(submit['predicted_score'].mean())
submit.to_csv('../../Submission/advertisement/lgb_trick_cross_offline_textbase_adj_502.txt', sep=" ", index=False, line_terminator='\n')

0.036213281019219425


In [16]:
submit['predicted_score'] = score_change(submit['predicted_score'],y_finnal_.mean(),0.0359194123126834)
print(submit['predicted_score'].mean())

0.03841031999485694


In [ ]:
submit = pd.read_csv('../../Submission/advertisement/gbm_trick_text_417.txt',sep=" ")